In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch

from Configs import getEnvConfig, visualizeEnvConfig, getPredictorConfig, visualizePredictorConfig
from EnvLibs import PolicySimulator, createEnv
from DiffusionLibs import DiffusionPolicyInterface
from MdpSolverLibs import MdpKernel

In [2]:
configIdx = 0
envParams = getEnvConfig(configIdx)
visualizeEnvConfig(envParams)
predictorParams = getPredictorConfig(configIdx)
visualizePredictorConfig(predictorParams)
trafficDataParentPath = f'Results/TrafficData'
env = createEnv(envParams, trafficDataParentPath)
env.selectMode(mode="test", type="data")

Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_fr
N_aggregation:          4
Resource Bar:           5
Bandwidth:              100
M List:                 [3, 4, 5]
Random Seed:            999
Alpha Range:            (0.01, 1.0)
Discrete Alpha Steps:   10
Predictor Configuration
Window Length:          200
Upsample K:             10
Dataflow:               thumb_fr
DB Parameter:           0.001
Alpha:                  0.01
Mode:                   fixed
Direction:              forward
Train Ratio:            0.6
Train Data Augment:     False
Smooth Fc:              1.5
Smooth Order:           3


KeyboardInterrupt: 

In [ ]:
diffusionQParams = {
    'device': 'cuda',
    'gamma': 0.1,
    'tau': 0.005,
    'lr': 5e-3,
    'eta': 1.0,
    'iql_tau': 0.95,
    'temperature': 3.0,
    'N_action_candidates': 50
}
diffusionPolicyInterface = DiffusionPolicyInterface(envParams, **diffusionQParams)

from tqdm import tqdm
for i in tqdm(range(500), desc="Sampling"):
    s = torch.tensor([[50,50,50,50]], device='cuda',  dtype=torch.float32) 
    diffusionPolicyInterface.diffusionQ.sample(s)

Sampling:  10%|▉         | 48/500 [00:06<01:01,  7.33it/s]


KeyboardInterrupt: 

In [ ]:
num_windows = 1000
obvMode = "predicted" # "perfect" or "predicted"
sample_method = "greedy"
# ==== Prepare DiffusionQ Policy====
with open(f'Results/DiffusionQ/diffusionQ_params_config{configIdx}.pkl', 'rb') as f:
    diffusionQParams = pickle.load(f)
diffusionPolicyInterface = DiffusionPolicyInterface(envParams, **diffusionQParams)
model_state_dict = torch.load(f'Results/DiffusionQ/diffusionQ_config{configIdx}.pth')
diffusionPolicyInterface.diffusionQ.load_state_dict(model_state_dict)
diffusionPolicyInterface.diffusionQ.eval()

C:\Users\Ye\AppData\Local\Temp\ipykernel_26396\294264270.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state_dict = torch.load(f'Results/DiffusionQ/diffusionQ_co

DiffusionQLearner(
  (diffusion_policy): EfficientDiffusionPolicy(
    (eps_net): Sequential(
      (0): Linear(in_features=142, out_features=64, bias=True)
      (1): Mish()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Mish()
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): Mish()
      (6): Linear(in_features=64, out_features=64, bias=True)
      (7): Mish()
      (8): Linear(in_features=64, out_features=10, bias=True)
    )
  )
  (diffusion_policy_target): EMATarget(
    (source): EfficientDiffusionPolicy(
      (eps_net): Sequential(
        (0): Linear(in_features=142, out_features=64, bias=True)
        (1): Mish()
        (2): Linear(in_features=64, out_features=64, bias=True)
        (3): Mish()
        (4): Linear(in_features=64, out_features=64, bias=True)
        (5): Mish()
        (6): Linear(in_features=64, out_features=64, bias=True)
        (7): Mish()
        (8): Linear(in_features=64, out_features=10, bias=True)
   

In [ ]:
env.reset()
for i in range(10):
    env.updateStates()
    s, s_pred = env.getStates()

s = [50,50,50,50]
print(s)
for i in range(10):
    a = diffusionPolicyInterface.sample(s, N_action_candidates=1000, sample_method="mean")
    print(a)



[50, 50, 50, 50]
build solver
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
(array([0, 0, 0, 0]), array([0., 0., 0., 0.], dtype=float32), 1, 1.0)
